In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# pandas options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [20]:
# spotify data 2023
spotify_data = pd.read_csv('data/spotify_top_10000_1960-now.csv')

In [21]:
spotify_data.isnull().sum()

Track URI                  0
Track Name                 1
Artist URI(s)              2
Artist Name(s)             1
Album URI                  2
Album Name                 1
Album Artist URI(s)        2
Album Artist Name(s)       2
Album Release Date         2
Album Image URL            4
Disc Number                0
Track Number               0
Track Duration (ms)        0
Track Preview URL       2897
Explicit                   0
Popularity                 0
ISRC                       3
Added By                   0
Added At                   0
Artist Genres            550
Danceability               2
Energy                     2
Key                        2
Loudness                   2
Mode                       2
Speechiness                2
Acousticness               2
Instrumentalness           2
Liveness                   2
Valence                    2
Tempo                      2
Time Signature             2
Album Genres            9999
Label                      6
Copyrights    

In [22]:
# drop rows with missing Artist Genres
spotify_data = spotify_data.dropna(subset=['Artist Genres'])

In [23]:
# convert track duration to seconds
spotify_data['Track Duration (ms)'] = spotify_data['Track Duration (ms)'] / 1000

In [24]:
# drop all columns containing 'URL' or 'URI'
spotify_data = spotify_data[spotify_data.columns.drop(list(spotify_data.filter(regex='URL')))]
spotify_data = spotify_data[spotify_data.columns.drop(list(spotify_data.filter(regex='URI')))]
# drop ISRC, Added By, Added At, Disc Number, Copyrights, Album Genres
spotify_data = spotify_data.drop(columns=['ISRC', 'Added By', 'Added At', 'Disc Number', 'Copyrights', 'Album Genres'])

In [ ]:
# create categorical buckets for Popularity (0-20 = 1, 21-40 = 2, 41-60 = 3, 61-80 = 4, 81-100 = 5)

In [25]:
spotify_data

,Track Name,Artist Name(s),Album Name,Album Artist Name(s),Album Release Date,Track Number,Track Duration (ms),Explicit,Popularity,Artist Genres,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Label
0,Justified & Ancient - Stand by the Jams,The KLF,Songs Collection,The KLF,1992-08-03,3,216.270,False,0,"acid house,ambient house,big beat,hip house",0.617,0.872,8.0,-12.305,1.0,0.0480,0.015800,0.112000,0.4080,0.504,111.458,4.0,Jams Communications
1,I Know You Want Me (Calle Ocho),Pitbull,Pitbull Starring In Rebelution,Pitbull,2009-10-23,3,237.120,False,64,"dance pop,miami hip hop,pop",0.825,0.743,2.0,-5.995,1.0,0.1490,0.014200,0.000021,0.2370,0.800,127.045,4.0,Mr.305/Polo Grounds Music/J Records
2,From the Bottom of My Broken Heart,Britney Spears,...Baby One More Time (Digital Deluxe Version),Britney Spears,1999-01-12,6,312.533,False,56,"dance pop,pop",0.677,0.665,7.0,-5.171,1.0,0.0305,0.560000,0.000001,0.3380,0.706,74.981,4.0,Jive
3,Apeman - 2014 Remastered Version,The Kinks,"Lola vs. Powerman and the Moneygoround, Pt. On...",The Kinks,2014-10-20,11,233.400,False,42,"album rock,art rock,british invasion,classic r...",0.683,0.728,9.0,-8.920,1.0,0.2590,0.568000,0.000051,0.0384,0.833,75.311,4.0,Sanctuary Records
4,You Can't Always Get What You Want,The Rolling Stones,Let It Bleed,The Rolling Stones,1969-12-05,9,448.720,False,0,"album rock,british invasion,classic rock,rock",0.319,0.627,0.0,-9.611,1.0,0.0687,0.675000,0.000073,0.2890,0.497,85.818,4.0,Universal Music Group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,Kernkraft 400 (A Better Day),"Topic, A7S",Kernkraft 400 (A Better Day),"Topic, A7S",2022-06-17,1,165.800,False,79,"german dance,pop dance,pop edm,uk dance,pop da...",0.623,0.727,11.0,-5.570,0.0,0.0562,0.184000,0.000020,0.3090,0.400,125.975,4.0,Virgin
9995,Never Say Never - Radio Edit,Vandalism,Never Say Never,Vandalism,2005-10-24,1,176.640,False,17,"australian dance,melbourne bounce",0.720,0.841,9.0,-6.373,1.0,0.0340,0.000354,0.011200,0.3380,0.767,130.978,4.0,Vicious
9996,Groovejet (If This Ain't Love) [feat. Sophie E...,"Spiller, Sophie Ellis-Bextor",Groovejet (If This Ain't Love) [feat. Sophie E...,Spiller,2000-08-14,1,227.619,False,62,"disco house,vocal house,dance pop,europop,new ...",0.719,0.806,9.0,-6.802,0.0,0.0389,0.000132,0.088900,0.3610,0.626,123.037,4.0,Defected Records
9997,Lay Low,Tiësto,Lay Low,Tiësto,2023-01-06,1,153.442,False,87,"big room,brostep,dutch edm,edm,house,pop dance...",0.534,0.855,1.0,-4.923,0.0,0.1830,0.060700,0.000263,0.3460,0.420,122.060,4.0,Musical Freedom
